# Tarea 2

Diego Lozoya Morales | 745345

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
from IPython.display import display

pd.set_option('display.float_format', '{:,.2f}'.format)

## Ejercicio 1 

Función VaR y ES

In [2]:
def VaR_CVaR(data, n_stocks, conf, long):
     
     valor_posicion = n_stocks * data.iloc[-1]
     weights = [valor_posicion.iloc[i]/valor_posicion.sum() for i in range(len(n_stocks))]  
     rt_assets = data.pct_change().dropna()
     rt_port = rt_assets.mul(weights).sum(axis=1)

     var = np.percentile(rt_port,100-conf) if long else np.percentile(rt_port,conf)
     cvar = rt_port[rt_port < var].mean() if long else rt_port[rt_port > var].mean()
          
     var_m = var * valor_posicion.sum()
     cvar_m = cvar * valor_posicion.sum()
     
     resultados_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var * 100, cvar * 100], "Monetario ($)": [var_m, cvar_m]})

     return resultados_df

In [3]:
data = yf.download(['AAPL', 'AMD', 'JPM', 'LMT', 'TSLA'], start="2020-01-01", end="2023-01-01")['Close'] #No puedo descargar el ajustado

positions = [2193, 3221, 1231, 761, 1211] #Ordenar alfabeticamente

long = True #True o False

conf = 95 #[0, 100]

VaR_CVaR(data, positions, conf, long)

[*********************100%***********************]  5 of 5 completed


,Métrica,Porcentual (%),Monetario ($)
0,VaR,-2.88,"-33,073.89"
1,CVaR,-4.63,"-53,131.46"


## Ejercicio 2

Rebalanceo del portafolio

In [4]:
rt = data.pct_change().dropna()
position_value = positions * data.iloc[-1]
portfolio_value = position_value.sum()
w_0 = position_value / portfolio_value
target_VaR = 30000 

In [5]:
def opt_weights(weights):
     
     def objective(weights):

          return abs(np.percentile(rt.dot(weights), 100 - conf)) if long else abs(np.percentile(rt.dot(weights), conf))
     
     constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}, {'type': 'eq', 'fun': lambda weights: objective(weights) - target_VaR})
     bounds = [(0, 1) for _ in range(len(positions))]
     initial_guess = np.full(len(positions), 1/len(positions))

     result = minimize(fun=objective, constraints=constraints, bounds=bounds, x0=initial_guess, method='SLSQP')
     
     return result.x

In [6]:
def rebalanceo(initial_weights, optimal_weights):
     
     df = pd.DataFrame({"Initial Weights": initial_weights*100, "Optimal Weights": optimal_weights*100, "Stocks difference": np.floor((optimal_weights - initial_weights) * portfolio_value / data.iloc[-1])})

     df['Buy/Sell'] = df['Stocks difference'].apply(lambda x: 'Buy' if x > 0 else ('Sell' if x < 0 else 'NA')) if long else df['Stocks difference'].apply(lambda x: 'Sell' if x > 0 else ('Buy' if x < 0 else 'NA'))
     
     return df

In [7]:
rebalanceo(w_0, opt_weights(w_0))

,Initial Weights,Optimal Weights,Stocks difference,Buy/Sell
Ticker,,,,
AAPL,24.57,0.00,"-2,193.00",Sell
AMD,18.20,35.40,"3,045.00",Buy
JPM,13.59,39.97,"2,390.00",Buy
LMT,30.64,14.53,-401.00,Sell
TSLA,13.01,10.11,-271.00,Sell


## Ejercicio 3

Investigación conceptual: Rebalanceos

### Inciso a)

- **¿Cuál es el objetivo principal del rebalanceo?**

Ayudar a los inversionistas a aliniarse con sus objetivos iniciales y preferencias de riesgo. Con el tiempo, las poneraciónes iniciales del portafolio van cambiando, aumentando el porcentaje de activos con mayor volatilidad por lo que, el portafolio, se debe de ajustar ya sea al propósito inicial del inversionista, o a nuevas necesidades, en caso de ser necesario.

- **¿Por qué es importante implementarlo de manera sistemática?**

Se puede dividir en dos grandes razones. La primera es para lograr mantener un nivel de exposición al riesgo deseado, evitando la concentración en activos volatiles. La segunda es para controlar en cierto modo las emociones de los inversionistas, y evitar que actúen de manera impulsiva, fomentando el cambio de ponderaciones en el activo únicamente bajo ciertos indicadores, evitando el guiarse por el reciente desempeño dle mercado y comprar caro para posteriormente vender barato.


### Inciso b)

- **¿Qué decisiones forman parte de una estrategia de rebalanceo?**

Una de las deciciones basicas es la periodicidad copn la que se debería de balancear el portafolio, sin embargo, esto depende bastante de cada inversionista, ya que probalemente hay personas que no cuentan con el tiempo necesario para realizarlo en el momento que les gustaría. De igual manera esto se ve influenciado por el propósito de inversión, de modo que no todas las personas logran invertir el mimso capital, por lo que no cuentan con la misma aversión al riesgo y su objetivo monetario es distinto.

- **¿Cuáles son los métodos principales para realizar rebalanceos de forma sistemática?**

     - **Rebalanceo basado en calendario:** Definir un periodo de tiempo, en el cual se cambiarian las ponderaciones de los activos de manera regular, por ejemplo cada 3, 6, 12 meses

     - **Rebalanceo basado en límite:** Definir un indicador el cual al ser rebasado deberá rebalancearse el portafolio, sin importar el momento en que esto suceda. 

     - **Rebalanceo basado en calendario y en límite:** Una combianción de ambas metodologías. Se establece un periodo de observación. En ese periodo se evalua el indicador y si es superado se rebalancea. Tiene que pasar ambvas cosas a la vez, que sea la fecha para checar y que se haya rebasado al indicador para poder rebalancear al portafolio.


### Inciso c)

- **Además del rendimiento, riesgo y objetivos, ¿qué otro factor puede tener un gran impacto en el desempeño de un portafolio al rebalancearlo?**

Cuestiones sobre impuestos y comisiones. Estas aunque son por lo general un porcentaje fijo, claramente aumentan conforme el número de operaciones, por lo que se recomienda al inversor promnedio que la periodicidad optima promedio del rebalanceo debe rondar un año. Puede ser que haya gente que lo necesite hacer más y menos frecuentemente, pero debe oscilar al rededor de un año. 


### Inciso d)

- **¿Qué recomendaciones propone Vanguard para reducir los costos fiscales en un portafolio?**

Recomienda hacer rebalanceos parciales en lugar de completos, teniendo un enfoque en acciones con un elevado costo en términos fiscales o en las clases de activos que están muy por encima o por debajo de su peso óptimo.

- **¿Qué es el tax loss harvesting?**

Es una estrategia de inversión utilizada para minimizar la carga fiscal, en donde generalmente se vende a los activos de menor rendimiento para cubrir la proporción atribuible a las ganancias de capital, de modo que se tengan que pagar menos impuestos.
